In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import matplotlib.pyplot as plt
import matplotlib.image as maping
import seaborn as sns

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

mnist_train = pd.read_csv("../input/Kannada-MNIST/train.csv")

mnist_test = pd.read_csv("../input/Kannada-MNIST/test.csv")

Y_train = mnist_train["label"]

X_train = mnist_train.drop(labels = ["label"], axis=1)

mnist_test = mnist_test.drop(labels = ["id"], axis=1)

X_train = X_train/255.0
mnist_test = mnist_test/255.0

X_train = X_train.values.reshape(-1,28,28,1)

Y_train = to_categorical(Y_train, num_classes=10)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=2)

model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(10, activation = "softmax"))

model.summary()

optimizer = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=200, verbose=2)

results = model.predict(mnist_test.values.reshape(-1,28,28,1)) 

results = np.argmax(results,axis = 1)

results = pd.Series(results,name="label")

submission = pd.concat([pd.Series(range(0,mnist_test.shape[0]),name = "id"),results],axis = 1)

submission.to_csv("submission.csv",index=False)